In [ ]:
%matplotlib ipympl

# BEER McStas data reduction

In [ ]:
import scipp as sc
import scippneutron as scn

from ess.beer import BeerMcStasWorkflow, BeerMcStasWorkflowKnownPeaks
from ess.beer.data import mcstas_silicon_medium_resolution, mcstas_duplex, duplex_peaks_array, silicon_peaks_array
from ess.reduce.nexus.types import Filename, SampleRun
from ess.reduce.time_of_flight.types import DetectorTofData
from ess.beer.types import *

dspacing = sc.linspace('dspacing', 0.8, 2.2, 2000, unit='angstrom')

In [ ]:
%%time
wf = BeerMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(9)
da = wf.compute(DetectorTofData[SampleRun])
da.hist(dspacing=dspacing).plot()

In [ ]:
%%time
wf = BeerMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(9)
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
p.ax.vlines(duplex_peaks_array().values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
p

In [ ]:
%%time
wf = BeerMcStasWorkflowKnownPeaks()
wf[DHKLList] = silicon_peaks_array()
wf[Filename[SampleRun]] = mcstas_silicon_medium_resolution()
da = wf.compute(DetectorTofData[SampleRun])
da.hist(dspacing=dspacing).plot()

In [ ]:
%%time
wf = BeerMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_silicon_medium_resolution()
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
p.ax.vlines(silicon_peaks_array().values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
p

In [ ]:
da['bank2'].bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

## Automatic peak finder and reduction for different chopper modes

### Mode 7

In [ ]:
wf = BeerMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(7)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
%%time
wf[MaxTimeOffset] = sc.scalar(6e-4, unit='s')
wf[MinTimeToNextStreak] = sc.scalar(8e-4/10, unit='s')
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
p.ax.vlines(duplex_peaks_array().values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
p

### Mode 8

In [ ]:
wf = BeerMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(8)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
wf[MaxTimeOffset] = sc.scalar(6e-4, unit='s')
wf[MinTimeToNextStreak] = sc.scalar(8e-4/10, unit='s')
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
p.ax.vlines(duplex_peaks_array().values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
p

### Mode 9

In [ ]:
wf = BeerMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(9)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
p.ax.vlines(duplex_peaks_array().values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
p

In [ ]:
da = da['bank2'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')
da.bins.masks['too_close_to_other'] = ~da.bins.masks.pop('too_close_to_other')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

### Mode 10

In [ ]:
wf = BeerMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(10)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
wf[MaxTimeOffset] = sc.scalar(3e-4/2, unit='s')
wf[MinTimeToNextStreak] = sc.scalar(8e-4/2, unit='s')
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
p.ax.vlines(duplex_peaks_array().values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
p

In [ ]:
da = da['bank1'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')
da.bins.masks['too_close_to_other'] = ~da.bins.masks.pop('too_close_to_other')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

### Mode 16

In [ ]:
wf = BeerMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(16)
wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

In [ ]:
wf[MaxTimeOffset] = sc.scalar(3e-4/2, unit='s')
wf[MinTimeToNextStreak] = sc.scalar(8e-4/2, unit='s')
da = wf.compute(DetectorTofData[SampleRun])
da = da.transform_coords(('dspacing',), graph=scn.conversion.graph.tof.elastic('tof'),)
p = da.hist(dspacing=dspacing, dim=da.dims).plot()
p.ax.vlines(duplex_peaks_array().values, 0, 20000, linestyle='--', color='black', lw=0.5, label='true $d_{hkl}$')
p

In [ ]:
da = da['bank1'].copy()
da.masks.clear()
da.bins.masks['too_far_from_center'] = ~da.bins.masks.pop('too_far_from_center')
da.bins.masks['too_close_to_other'] = ~da.bins.masks.pop('too_close_to_other')

da.bins.concat().hist(two_theta=1000, event_time_offset=1000).plot(norm='log')

## Reuction with known peak workflow

## Mode 7

In [ ]:
wf = BeerMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(7)
da = wf.compute(DetectorTofData[SampleRun])
da.hist(dspacing=dspacing).plot()

## Mode 8

In [ ]:
wf = BeerMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(8)
da = wf.compute(DetectorTofData[SampleRun])
da.hist(dspacing=dspacing).plot()

## Mode 9

In [ ]:
wf = BeerMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(9)
da = wf.compute(DetectorTofData[SampleRun])
da.hist(dspacing=dspacing).plot()

## Mode 10

In [ ]:
wf = BeerMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Filename[SampleRun]] = mcstas_duplex(10)
da = wf.compute(DetectorTofData[SampleRun])
da.hist(dspacing=dspacing).plot()

## Mode 16

In [ ]:
wf = BeerMcStasWorkflowKnownPeaks()
wf[DHKLList] = duplex_peaks_array()
wf[Time0] = sc.scalar(0.5 * 17/360 /28, unit='s')
wf[Filename[SampleRun]] = mcstas_duplex(16)
da = wf.compute(DetectorTofData[SampleRun])
da.hist(dspacing=dspacing).plot()